<a href="https://colab.research.google.com/github/ChKik/anaktisipliroforias/blob/main/AnaktisiProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  #mount to drive prwta gia na kanei locate ta file meta

In [ ]:
#Load ta data se ena morfhs tsv arxeio gia na mporei to colbert na ta diaxeiristei meta

from collections import defaultdict
import csv

null = None
true = True
false = False


#Functions


# File paths
relevDocs_path = '/content/drive/MyDrive/Anaktisi/Relevant_20'
detailedFile_path = '/content/drive/MyDrive/Anaktisi/cfquery_detailed'
csvCombined = '/content/drive/MyDrive/Anaktisi/combined_data.csv'

# Variables and data storage
queriesAnswerDocs = defaultdict(dict)
temp_qid = null

# Read the data from the files
with open(detailedFile_path, 'r') as qfile:
    fileElements = qfile.readlines()

with open(relevDocs_path, 'r') as relevDocsFile:
    relevant_docs = relevDocsFile.readlines()

iterator = iter(relevant_docs)

############

for x in fileElements:
    try:
        if x.startswith('QN'):
            if temp_qid is not null:
                queriesAnswerDocs[temp_qid]['RD'] = doc_id  # Yparxei QN to vazei sto antistoixo the doc ID

            temp_qid = int(x.split()[1])  # pairnei to query ID me morfh QN 000

            line=next(iterator)
            doc_id = [int(doc_id) for doc_id in line.split()]

        elif x.startswith('NR'):
            queriesAnswerDocs[temp_qid] = {'NR': int(x.split()[1]), 'RD': []}  # Initialize ta NR kai RD
        elif x.startswith(''):  #agnoei tis grammes poy einai kenes stin arxh
          continue
        else:
            raise ValueError("Unexpected line format in fileElements")

    except StopIteration:
        print(f"Error: No more lines in relevant_docs to process for query {temp_qid}")
        break

    except ValueError as ve:
        print(f"Value error occurred: {ve}")
        exit(1)

    except Exception as e:
        print(f"An error occurred: {e}")
        exit(1)


#vazei sto query 20 poy einai to teleutaio ,ta stoixeia poy toy antistixoyn.
if temp_qid is not None:
    print(f'Total queries: {temp_qid}')
    queriesAnswerDocs[temp_qid]['RD'] = doc_id


with open(csvCombined, 'w', newline='') as csv_file:
    record_data = csv.writer(csv_file)

    # record_data header
    record_data.writerow(['Query_ID', 'NR', 'RD'])

    # Write data
    for query_id, data in queriesAnswerDocs.items():
        record_data.writerow([query_id, data['NR'], data['RD']])

#printer
with open(csvCombined, 'r', newline='') as csv_file:  #csv me tab seperated ara ginetai tsv
      csv_reader = csv.reader(csv_file)

      for x in csv_reader:
          print(x)


In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

try: #When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False


In [4]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [5]:
import os

docs_path = '/content/drive/MyDrive/Anaktisi/docs'
resultDocs_path = '/content/drive/MyDrive/Anaktisi/resultDocs.tsv'

resultDocsList=[]

with open(resultDocs_path, 'a', encoding='utf-8') as tsv_file:
  for filename in os.listdir(docs_path):
    full_path = os.path.join(docs_path, filename)  #pairnei to path toy kathe doc ksexwrista
    if os.path.isfile(full_path):
        try:
            with open(full_path, 'r', encoding='utf-8') as temp_file:
                document_content = temp_file.read()
                resultDocsList.append(document_content)
                tsv_file.write(document_content.replace('\n', ' ') + '\n')
        except Exception as e:
            print(f"Error processing document {filename}: {e}")
    else:
        print(f"{filename} is not a file,its a directory.")



In [6]:
query_path = '/content/drive/MyDrive/Anaktisi/Queries_20'
queriesList = []


with open(query_path, 'r') as f:
    queries = f.read().splitlines()
    queriesList.append(queries)

numOfDocs = len(os.listdir(docs_path))


print(f'Loaded {len(queriesList[0])} queries and {numOfDocs:,} docs')  #bgazei 19,leipei to teleutaio query apo tin Queries_20,den kserw ama einai epithdes etsi.



Loaded 19 queries and 1,209 docs


In [ ]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 2000  #max docs 2000
datasplit='dev'
dataset='kikidis'

index_name = f'{dataset}.{datasplit}.{nbits}bits'

checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=resultDocsList[:max_id], overwrite=True)  #resultDocsList gia ton indexer.




In [ ]:
indexer.get_index()
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=resultDocsList)


In [ ]:
import pandas as pd

query="What is the lipid composition of CF respiratory secretions"  #mporw na to kanw kai random me map h epilogh kapoiou alla den exei kai poly nohma, to vazw statiko gia test
query_num=4

docRetriaval=pd.read_csv('/content/drive/MyDrive/Anaktisi/combined_data.csv')

queryInfo=docRetriaval[docRetriaval['Query_ID'] == query_num].iloc[0] #tha epistrepsei to prwto antikeimeno stin thesh 0 gia to query ayto.
NR=queryInfo['NR']
RD=eval(queryInfo['RD'])
final_results=searcher.search(query,k=NR)

possRelevantDocs = [doc_id for doc_id, doc_rank, doc_score in zip(*final_results)]
actual_docs = len(set(possRelevantDocs) & set(RD))

if len(possRelevantDocs) > 0:
  precision = actual_docs / len(possRelevantDocs)
else:
  precision=0.0

if len(RD) > 0:
  recall = actual_docs / len(RD)
else:
  recall= 0.0

print(f"\t Rank \t\tScore \t\t DocumentID")
for doc_id,doc_rank,doc_score in zip(*final_results):
    print(f"\t [{doc_rank}] \t\t {doc_score:.3f} \t\t {doc_id}")

print("Query Evaluation Measures:")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")